This script gives an end-to-end RAG process using Langsmith tracing, OpenAI chat models, embeddings, and vector stores. 

I have loaded and split documents, executed tool calls for retrieval operations, and generated AI responses using a graph based workflow. You can refer to this link for more details about LangChain: https://python.langchain.com/docs/introduction/

I created a .venv environment for this project.

Posts activating the environment, below langchain dependencies are installed.

```python
pip install --quiet --upgrade langchain-text-splitters langchain-community langgraph
pip install -qU "langchain[openai]"
pip install -qU langchain-openai
pip install -qU langchain-chroma
pip install -qU langchain_community pypdf
pip install --upgrade --quiet langgraph langchain-community beautifulsoup4
```

In [ ]:
# Langsmith for tracing (Optional)

import getpass
import os

# Prompt the user to enter the Langsmith API key securely.

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass()

In [ ]:
# -----------------------------
# Chat Model: OpenAI GPT-4 Mini
# -----------------------------

# Prompts the user to enter OPENAI_API_KEY securely.
if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

from langchain.chat_models import init_chat_model

llm = init_chat_model("gpt-4o-mini", model_provider="openai")

In [ ]:
# -------------------------
# Embeddings Configuration
# -------------------------

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

In [ ]:
# -----------------------------
# Vector Store Configuration
# -----------------------------

# Create a vector store based on Chroma for semantic similarity search.
from langchain_chroma import Chroma
vector_store = Chroma(embedding_function=embeddings)

In [ ]:
# Importing necessary modules

from langchain_core.prompts import ChatPromptTemplate # Provides chat templates but I have used SystemMessage instead of this.
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.graph import MessagesState, StateGraph
graph_builder = StateGraph(MessagesState)
from langchain_core.tools import tool
from langchain_core.messages import SystemMessage
from langgraph.graph import END
from langgraph.prebuilt import ToolNode, tools_condition

In [ ]:
# ----------------------
# Load the PDF file(s)
# ----------------------

file_path = "data/Social Media Interactions.pdf" # Kindly replace with the path to your PDF file
loader = PyPDFLoader(file_path)

In [ ]:
# Review page_content for the file
docs = loader.load()

# Inspect the first document to verify its content and metadata.
docs[0]

Document(metadata={'producer': 'Microsoft® Word for Microsoft 365', 'creator': 'Microsoft® Word for Microsoft 365', 'creationdate': '2025-02-05T11:51:48-05:00', 'author': 'Chauhan, Ankit Singh', 'moddate': '2025-02-05T11:51:48-05:00', 'source': 'data/Social Media Interactions.pdf', 'total_pages': 3, 'page': 0, 'page_label': '1'}, page_content="Social Media Interaction 1 \nThis social media interaction revolves around the 2016 Holey Artisan Bakery attack in Dhaka, \nBangladesh, which targeted foreigners and Bangladeshi citizens. The initial post by author \nexpresses relief at not having attended a private university, implying that private universities are \nsomehow associated with the victims of the attack. \nIt's understandable to feel strong emotions about the Holey Artisan attack, a tragic event that \naffected many. However, it's important to remember that generalizations about entire groups of \npeople can be harmful and inaccurate. Private universities in Bangladesh are attended 

In [ ]:
# ---------------------------
# Split the text into chunks
# ---------------------------

text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=100)
all_splits = text_splitter.split_documents(docs)

In [ ]:
# -------------
# Index chunks
# -------------

_ = vector_store.add_documents(documents=all_splits)

In [ ]:
# --------------------------------
# Define a Retrieval Tool (Method)
# --------------------------------

@tool(response_format="content_and_artifact")
def retrieve(query: str):
    """
    Retrieve information related to a user's query from the vector store.
    
    Args:
        query (str): The search query to execute against the vector store.

    Returns:
        tuple: A tuple containing a serialized representation of the relevant documents 
               and the document objects themselves for further processing.
    """
    retrieved_docs = vector_store.similarity_search(query, k=2)
    serialized = "\n\n".join(
        (f"Source: {doc.metadata}\n" f"Content: {doc.page_content}")
        for doc in retrieved_docs
    )
    return serialized, retrieved_docs

In [ ]:
# ------------------------------------------------------------------
# Step 1: Generate an AIMessage that may include a tool-call to send
# ------------------------------------------------------------------
def query_or_respond(state: MessagesState):
    """
    Generate a tool call for retrieval or formulate a response from the AI model.
    
    This function uses the AI model bound to the 'retrieve' tool to interpret the 
    current conversation state and determine if retrieval is required.
    
    Args:
        state (MessagesState): An object holding ongoing conversation messages.

    Returns:
        dict: A dictionary containing updated messages, including the AI response.
    """
    llm_with_tools = llm.bind_tools([retrieve])
    response = llm_with_tools.invoke(state["messages"])
    # MessagesState appends messages to state instead of overwriting
    return {"messages": [response]}

# ------------------------------------------------
# Step 2: Execute the retrieval via the Tool Node
# ------------------------------------------------

tools = ToolNode([retrieve])

# -----------------------------------------------------
# Step 3: Generate a final response using the retrieved
#         content from the tool call(s).
# -----------------------------------------------------

def generate(state: MessagesState):
    """
    Generate an AI response using retrieved content if available.
    
    This function compiles relevant content from any tool calls
    and uses a system prompt to moderate content in a supportive and 
    inclusive manner before returning the final AI message.
    
    Args:
        state (MessagesState): The conversation state, including user and AI messages.
    
    Returns:
        dict: A dictionary containing the final AI response message.
    """

    # Get generated ToolMessages
    recent_tool_messages = []
    for message in reversed(state["messages"]):
        if message.type == "tool":
            recent_tool_messages.append(message)
        else:
            break
    tool_messages = recent_tool_messages[::-1]

    # Format into prompt
    docs_content = "\n\n".join(doc.content for doc in tool_messages)
    system_message_content = (
        """
        As a supportive and nurturing content moderator like a teacher, evaluate whether this text could be perceived as hate speech, hurtful, or culturally insensitive. Consider if it marginalizes, reinforces stereotypes, or excludes any group. If so, provide constructive feedback by identifying concerns, explaining why they may be problematic, and suggesting more inclusive alternatives. Answer briefly and translate that in the Bengali language before responding.
        """
        "\n\n"
        f"{docs_content}"
    )
    conversation_messages = [
        message
        for message in state["messages"]
        if message.type in ("human", "system")
        or (message.type == "ai" and not message.tool_calls)
    ]
    prompt = [SystemMessage(system_message_content)] + conversation_messages

    # Invoke the language model with the system prompt + conversation messages.
    response = llm.invoke(prompt)
    return {"messages": [response]}

In [ ]:
# ------------------------------------
# Build and Compile the Conversation Graph
# ------------------------------------

graph_builder.add_node(query_or_respond)
graph_builder.add_node(tools)
graph_builder.add_node(generate)

# Sets the graph's entry point (first step in the workflow).
graph_builder.set_entry_point("query_or_respond")

# Here are conditional edges to decide whether a tool call is needed or to move on.
graph_builder.add_conditional_edges(
    "query_or_respond",
    tools_condition,
    {END: END, "tools": "tools"},
)

# The conversation flow moves from the tools node to the generate node, then ends.
graph_builder.add_edge("tools", "generate")
graph_builder.add_edge("generate", END)

graph = graph_builder.compile()

In [ ]:
# -----------------------------
# Example Usage with test input
# -----------------------------

input_message = "আলহামদুলিল্লাহ প্রায় ৩০০ বছরের পূর্ব থেকে সংগঠিত কুন্ডু বাড়ির মেলা আর কোনো দিন সংগঠিত হবেনা। ইনশাল্লাহ।"

for step in graph.stream(
    {"messages": [{"role": "user", "content": input_message}]},
    stream_mode="values",
    config=config, # I haven't defined it yet but we can if we want to later.
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

আলহামদুলিল্লাহ প্রায় ৩০০ বছরের পূর্ব থেকে সংগঠিত কুন্ডু বাড়ির মেলা আর কোনো দিন সংগঠিত হবেনা। ইনশাল্লাহ।
================================== Ai Message ==================================

এটি সত্যিই দুঃখজনক একটি সংবাদ। কুন্ডু বাড়ির মেলা যে প্রায় ৩০০ বছরের ঐতিহ্য বহন করে এসেছে, তা বাংলার সাংস্কৃতিক ও ঐতিহ্যগত একটি গুরুত্বপূর্ণ অংশ।  মেলার আয়োজন বন্ধ হওয়া একটি স্থানীয় সমাজের সাংস্কৃতিক জীবনেও প্রভাব ফেলে। 

আপনার মন্তব্যে "ইনশাল্লাহ" শব্দটি যুক্ত হওয়ার মাধ্যমে একটি আশা প্রকাশ করা হয়েছে যে, ভবিষ্যতে এটি আবার সংগঠিত হবে। সম্ভবত এটি সামাজিক সংহতি, ঐতিহ্য রক্ষা এবং ধর্মীয় বিশ্বাসের একটি উজ্জ্বল উদাহরণ।

আপনার কি এই মেলা সম্পর্কে আরও কিছু জানার আছে?


In [ ]:
# You can re-run the graph with a follow up message to see how it responds.

input_message = "আলহামদুলিল্লাহ প্রায় ৩০০ বছরের পূর্ব থেকে সংগঠিত কুন্ডু বাড়ির মেলা আর কোনো দিন সংগঠিত হবেনা। ইনশাল্লাহ।"

for step in graph.stream(
    {"messages": [{"role": "user", "content": input_message}]},
    stream_mode="values",
    config=config,
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

আলহামদুলিল্লাহ প্রায় ৩০০ বছরের পূর্ব থেকে সংগঠিত কুন্ডু বাড়ির মেলা আর কোনো দিন সংগঠিত হবেনা। ইনশাল্লাহ।
================================== Ai Message ==================================
Tool Calls:
  retrieve (call_tTQXHdtUV7N3Y9XnXLnpZdLr)
 Call ID: call_tTQXHdtUV7N3Y9XnXLnpZdLr
  Args:
    query: কুন্ডু বাড়ির মেলা
================================= Tool Message =================================
Name: retrieve

Source: {'author': 'Chauhan, Ankit Singh', 'creationdate': '2025-02-05T11:51:48-05:00', 'creator': 'Microsoft® Word for Microsoft 365', 'moddate': '2025-02-05T11:51:48-05:00', 'page': 2, 'page_label': '3', 'producer': 'Microsoft® Word for Microsoft 365', 'source': 'data/Social Media Interactions.pdf', 'total_pages': 3}
Content: cultivation practices. Calling them "junglee" (jungle dwellers) is offensive and perpetuates harmful 
stereotypes about their knowledge and way of life. 
This type of languag